In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/spm.model
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/config.json
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/pytorch_model.generator.bin
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/README.md
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/tf_model.h5
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/tokenizer_config.json
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/gitattributes
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/pytorch_model.bin
/kaggle/input/mdeberta-v3-base/transformers/default/1/mdeberta-v3-base/generator_config.json
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-cha

# Installing and Importing Libraries

In [2]:
!pip install -q --upgrade \
    transformers \
    tokenizers \
    datasets \
    evaluate \
    accelerate \
    torchmetrics

import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    get_linear_schedule_with_warmup,
)
from accelerate import Accelerator
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from torch import nn
from torch.optim import AdamW

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

2025-10-22 09:46:13.744644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761126373.932079      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761126373.987776      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")
sample_submission = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")
test_labels = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip")

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()


Train shape: (159571, 8)
Test shape: (153164, 2)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
# Count each toxic category
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

# Count how many 1s each column has
category_counts = train[label_cols].sum().sort_values(ascending=False)
print("✅ Count of each label (how many comments have this label):")
print(category_counts)

# Total number of positive labels across all columns
total_positive_labels = category_counts.sum()
print("\nTotal number of label instances (sum of all 6 columns):", total_positive_labels)

# Number of comments that have at least one label
rows_with_any_label = (train[label_cols].any(axis=1)).sum()
print("Number of comments with at least one label:", rows_with_any_label)

# Number of comments with no label at all
rows_with_no_label = len(train) - rows_with_any_label
print("Number of comments with no label:", rows_with_no_label)

# (Optional) Distribution of how many labels each comment has (0–6)
label_combo_counts = train[label_cols].sum(axis=1).value_counts().sort_index()
print("\nHow many labels each comment typically has (0–6):")
print(label_combo_counts)


✅ Count of each label (how many comments have this label):
toxic            15294
obscene           8449
insult            7877
severe_toxic      1595
identity_hate     1405
threat             478
dtype: int64

Total number of label instances (sum of all 6 columns): 35098
Number of comments with at least one label: 16225
Number of comments with no label: 143346

How many labels each comment typically has (0–6):
0    143346
1      6360
2      3480
3      4209
4      1760
5       385
6        31
Name: count, dtype: int64


In [5]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

# 1) Show dtype of each label column
print(train[label_cols].dtypes)

# 2) Unique values per label (first 50 unique entries) — good to spot non-binary tokens
for c in label_cols:
    vals = train[c].dropna().astype(str).str.strip().unique()[:50]
    print(f"\n{c} unique examples (up to 50):\n", vals)

# 3) Count NaNs per label
print("\nNaN counts per label:")
print(train[label_cols].isna().sum())

# 4) Count values other than '0' or '1' (string or numeric)
for c in label_cols:
    bad = train[~train[c].isin([0,1]) & ~train[c].isin(['0','1'])][c]
    print(f"{c} — non 0/1 sample count:", len(bad))
    if len(bad) > 0:
        print("Example non-binary values:", bad.astype(str).unique()[:20])

# 5) Rows where labels sum > 6 or < 0 (impossible) — shows unexpected numeric values
sums = train[label_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1)
print("\nRows with label-sum < 0 or > 6 (should be none):", ((sums<0) | (sums>6)).sum())

# 6) Distribution of #labels per row (read-only)
print("\nDistribution (0..6) of number of positive labels per row:")
print(train[label_cols].apply(lambda s: pd.to_numeric(s, errors='coerce')).sum(axis=1).value_counts().sort_index())

# 7) Duplicate checking by 'id' (if id exists) and by comment text
if 'id' in train.columns:
    print("\nDuplicate id count:", train['id'].duplicated().sum())
    print("Unique duplicate id examples (up to 10):", train[train['id'].duplicated(keep=False)]['id'].unique()[:10])
if 'comment_text' in train.columns:
    print("\nDuplicate comment_text count:", train['comment_text'].duplicated().sum())

# 8) Show sample rows that are labeled but look suspicious (e.g., non-binary)
mask_nonbinary = False
for c in label_cols:
    mask_nonbinary |= ~train[c].isin([0,1]) & ~train[c].isin(['0','1'])
if mask_nonbinary.any():
    print("\nExample rows with non-binary label values (first 10):")
    print(train[mask_nonbinary].head(10)[['id','comment_text'] + label_cols])
else:
    print("\nNo non-binary label values found in the subset checked.")

# 9) Check leading/trailing whitespace or invisible chars in label columns
# Show rows where stringified label differs from stripped version
def has_whitespace_issue(col):
    s = train[col].astype(str)
    return (s != s.str.strip()).any()
for c in label_cols:
    print(f"{c} has leading/trailing whitespace?:", has_whitespace_issue(c))


toxic            int64
severe_toxic     int64
obscene          int64
threat           int64
insult           int64
identity_hate    int64
dtype: object

toxic unique examples (up to 50):
 ['0' '1']

severe_toxic unique examples (up to 50):
 ['0' '1']

obscene unique examples (up to 50):
 ['0' '1']

threat unique examples (up to 50):
 ['0' '1']

insult unique examples (up to 50):
 ['0' '1']

identity_hate unique examples (up to 50):
 ['0' '1']

NaN counts per label:
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64
toxic — non 0/1 sample count: 0
severe_toxic — non 0/1 sample count: 0
obscene — non 0/1 sample count: 0
threat — non 0/1 sample count: 0
insult — non 0/1 sample count: 0
identity_hate — non 0/1 sample count: 0

Rows with label-sum < 0 or > 6 (should be none): 0

Distribution (0..6) of number of positive labels per row:
0    143346
1      6360
2      3480
3      4209
4      1760
5       385
6        

In [6]:
# Label columns
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y = train[label_cols]


In [7]:
import re

# Define cleaning patterns (same as found during EDA)
url_pattern = r"http\S+|www\.\S+"
html_pattern = r"<.*?>"
emoji_pattern = r"[\U00010000-\U0010ffff]"       # captures emojis and symbols
non_ascii_pattern = r"[^\x00-\x7F]+"             # captures non-ASCII chars
multi_space_pattern = r"\s+"                     # normalize spaces

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(url_pattern, " ", text)
    text = re.sub(html_pattern, " ", text)
    text = re.sub(emoji_pattern, " ", text)
    text = re.sub(non_ascii_pattern, " ", text)
    text = re.sub(multi_space_pattern, " ", text).strip()
    return text

# Apply cleaning to train and test sets
train["comment"] = train["comment_text"].apply(clean_text)
test["comment"] = test["comment_text"].apply(clean_text)

# Quick preview to ensure it worked
train[["comment_text", "comment"]].head(5)

,comment_text,comment
0,Explanation\nWhy the edits made under my usern...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,d'aww! he matches this background colour i'm s...
2,"Hey man, I'm really not trying to edit war. It...","hey man, i'm really not trying to edit war. it..."
3,"""\nMore\nI can't make any real suggestions on ...",""" more i can't make any real suggestions on im..."
4,"You, sir, are my hero. Any chance you remember...","you, sir, are my hero. any chance you remember..."


In [8]:
import random

for label in label_cols:
    sample_row = train[train[label] == 1].sample(1, random_state=random.randint(1,100))
    print(f"🔹 Category: {label}")
    print(sample_row['comment_text'].values[0])
    print("-" * 100)

🔹 Category: toxic
== ALL WIKI MODS A LIMPED-DICKED LIBTARD COWARD FUCKBAGS

http://en.wikipedia.org/wiki/User:Raul654 CAN GO FUCK HIMSELF, COWARD PIECE OF SHIT.

 ==
----------------------------------------------------------------------------------------------------
🔹 Category: severe_toxic
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS CENSORSHIP. WIKIPEDIA IS COMMUNISM.
WIKIPEDIA LOVES ME. THEY LIKE TO CENSOR ME. COMMUNISM IS 

In [9]:
test_df = test.drop(columns=['comment_text','id'])
test_df

,comment
0,yo bitch ja rule is more succesful then you'll...
1,"== from rfc == the title is fine as it is, imo."
2,""" == sources == * zawe ashton on lapland / """
3,":if you have a look back at the source, the in..."
4,i don't anonymously edit articles at all.
...,...
153159,". i totally agree, this stuff is nothing but t..."
153160,== throw from out field to home plate. == does...
153161,""" == okinotorishima categories == i see your c..."
153162,""" == """"one of the founding nations of the eu -..."


In [10]:
import pandas as pd

# Load the dataset
train = train.drop(columns=['id','comment_text' ])

train

,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment
0,0,0,0,0,0,0,explanation why the edits made under my userna...
1,0,0,0,0,0,0,d'aww! he matches this background colour i'm s...
2,0,0,0,0,0,0,"hey man, i'm really not trying to edit war. it..."
3,0,0,0,0,0,0,""" more i can't make any real suggestions on im..."
4,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember..."
...,...,...,...,...,...,...,...
159566,0,0,0,0,0,0,""":::::and for the second time of asking, when ..."
159567,0,0,0,0,0,0,you should be ashamed of yourself that is a ho...
159568,0,0,0,0,0,0,"spitzer umm, theres no actual article for pros..."
159569,0,0,0,0,0,0,and it looks like it was actually you who put ...


In [11]:
from sklearn.model_selection import train_test_split

# 1. Remove rows with missing or empty cleaned text
before = len(train)
train = train[train["comment"].notna() & (train["comment"].str.strip() != "")]
after = len(train)
print(f"Removed {before - after} rows with missing or empty cleaned text.")

# 2. Remove duplicate comments (based on comment)
before = len(train)
train = train.drop_duplicates(subset=["comment"])
after = len(train)
print(f"Removed {before - after} duplicate rows.")

# 3. Train–Validation split (80–20), stratified on "toxic" label
LABELS = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

train_df, val_df = train_test_split(
    train,
    test_size=0.2,
    random_state=42,
    stratify=train["toxic"] if "toxic" in train.columns else None
)

print("\nFinal sizes:")
print("Train:", train_df.shape)
print("Validation:", val_df.shape)

# Optional: Print label distribution in train and val sets
if all(col in train.columns for col in LABELS):
    print("\nTrain set label distribution:")
    print(train_df[LABELS].sum())
    print("\nValidation set label distribution:")
    print(val_df[LABELS].sum())

Removed 8 rows with missing or empty cleaned text.
Removed 316 duplicate rows.

Final sizes:
Train: (127397, 7)
Validation: (31850, 7)

Train set label distribution:
toxic            12197
severe_toxic      1298
obscene           6736
threat             390
insult            6253
identity_hate     1137
dtype: int64

Validation set label distribution:
toxic            3049
severe_toxic      292
obscene          1684
threat             84
insult           1597
identity_hate     260
dtype: int64


In [12]:
train_df

,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment
123716,0,0,0,0,0,0,arguement. yours was a personal attack (which ...
6376,0,0,0,0,0,0,""" speedy deletion of """"lilsprezy"""" a page you ..."
36674,0,0,0,0,0,0,""" spamstar spamstar the spamstar of glory pres..."
10290,0,0,0,0,0,0,i can't decide who's plagiarizing who the text...
43128,0,0,0,0,0,0,and page is getting 76k hits an hour as a result.
...,...,...,...,...,...,...,...
100831,0,0,0,0,0,0,hi! i will look at it. thanks.
60712,0,0,0,0,0,0,"stop redirecting, she should get her own page."
25297,0,0,0,0,0,0,"""==ga review== :this review is transcluded fro..."
151469,0,0,0,0,0,0,lot (real estate) what sort of wikifying do yo...


In [13]:

val_df


,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment
143614,0,0,0,0,0,0,""" delete hey there, do you want page deleted a..."
113762,0,0,0,0,0,0,"""::: to most the would appear mutually exclusi..."
152784,0,0,0,0,0,0,"i'm sorry, you may have misunderstood me. plea..."
35738,0,0,0,0,0,0,ryulong-you seem to bully people.
28510,0,0,0,0,0,0,""" the editor known as realist2 is a known trou..."
...,...,...,...,...,...,...,...
27809,0,0,0,0,0,0,why non-neutral? perhaps there is a tinge of n...
137851,0,0,0,0,0,0,are u the ultimate ruler of wiki i will never ...
80317,0,0,0,0,0,0,"""== which article is the best target for a """"w..."
13139,0,0,0,0,0,0,bias the citation link is broken - site linked...


In [14]:
test_df

,comment
0,yo bitch ja rule is more succesful then you'll...
1,"== from rfc == the title is fine as it is, imo."
2,""" == sources == * zawe ashton on lapland / """
3,":if you have a look back at the source, the in..."
4,i don't anonymously edit articles at all.
...,...
153159,". i totally agree, this stuff is nothing but t..."
153160,== throw from out field to home plate. == does...
153161,""" == okinotorishima categories == i see your c..."
153162,""" == """"one of the founding nations of the eu -..."


In [15]:
# ====================================================
# KAGGLE MULTI-GPU SETUP
# ====================================================
from accelerate import Accelerator
import torch
import numpy as np
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding, 
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW  # ✅ Import from torch.optim instead
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn as nn

# ====================================================
# CONFIG
# ====================================================
MODEL_PATH = "microsoft/mdeberta-v3-base"
NUM_LABELS = 6
EPOCHS = 6
BATCH_SIZE = 16  # Per GPU, effective = 32
LR = 2e-5
MAX_LEN = 256
SEED = 42
PATIENCE = 2
MIN_DELTA = 0.0005

LABELS = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

torch.manual_seed(SEED)
np.random.seed(SEED)

# ====================================================
# CHECK GPU AVAILABILITY
# ====================================================
print(f"🔧 GPUs available: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

# ====================================================
# TOKENIZER
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# ====================================================
# DATASET CLASS
# ====================================================
class ToxicDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256, is_test=False):
        self.texts = df["comment"].tolist()
        self.is_test = is_test
        self.tokenizer = tokenizer
        self.max_len = max_len
        if not is_test:
            self.labels = df[LABELS].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}

        if not self.is_test:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item


# ====================================================
# CREATE DATASETS
# ====================================================
train_dataset = ToxicDataset(train_df, tokenizer, max_len=MAX_LEN)
val_dataset = ToxicDataset(val_df, tokenizer, max_len=MAX_LEN)
test_dataset = ToxicDataset(test_df, tokenizer, max_len=MAX_LEN, is_test=True)

data_collator = DataCollatorWithPadding(tokenizer)

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=data_collator,
    num_workers=2,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=data_collator,
    num_workers=2,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=data_collator,
    num_workers=2,
    pin_memory=True
)

# ====================================================
# MODEL
# ====================================================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# ====================================================
# WEIGHTED LOSS
# ====================================================
label_sum = train_df[LABELS].sum(axis=0)
label_freq = label_sum / len(train_df)
class_weights = (1 / (label_freq + 1e-6))
class_weights = class_weights / class_weights.sum() * NUM_LABELS
class_weights_tensor = torch.tensor(class_weights.values, dtype=torch.float)

print("✅ Class Weights:", class_weights_tensor)

class WeightedBCEWithLogitsLoss(nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = weights

    def forward(self, logits, targets):
        loss = nn.functional.binary_cross_entropy_with_logits(
            logits, targets, weight=self.weights.to(logits.device)
        )
        return loss

criterion = WeightedBCEWithLogitsLoss(class_weights_tensor)

# ====================================================
# OPTIMIZER & SCHEDULER
# ====================================================
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)  # ✅ Now from torch.optim
num_training_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)

# ====================================================
# ✅ ACCELERATOR - AUTOMATICALLY USES BOTH GPUs
# ====================================================
accelerator = Accelerator(
    mixed_precision="fp16",
    gradient_accumulation_steps=1
)

# Prepare everything
train_loader, val_loader, test_loader, model, optimizer, scheduler = accelerator.prepare(
    train_loader, val_loader, test_loader, model, optimizer, scheduler
)

print(f"✅ Training on {accelerator.num_processes} GPU(s)")
print(f"✅ Effective batch size: {BATCH_SIZE * accelerator.num_processes}")

# ====================================================
# TRAINING LOOP WITH EARLY STOPPING
# ====================================================
best_auc = 0.0
patience_counter = 0
best_model_path = "best_model"

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    
    if accelerator.is_main_process:
        progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    else:
        progress = train_loader

    for batch in progress:
        with accelerator.accumulate(model):
            outputs = model(**batch)
            logits = outputs.logits
            loss = criterion(logits, batch["labels"])
            
            accelerator.backward(loss)
            
            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item()
        if accelerator.is_main_process:
            progress.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_loader)
    
    if accelerator.is_main_process:
        print(f"\n{'='*60}")
        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"{'='*60}")
        print(f"Train Loss: {avg_train_loss:.4f}")

    # ====================================================
    # VALIDATION
    # ====================================================
    model.eval()
    preds, refs = [], []
    val_loss = 0
    
    with torch.no_grad():
        for batch in val_loader:
            outputs = model(**batch)
            logits = outputs.logits
            
            loss = criterion(logits, batch["labels"])
            val_loss += loss.item()
            
            probs = torch.sigmoid(logits)
            all_probs = accelerator.gather(probs)
            all_labels = accelerator.gather(batch["labels"])
            
            preds.append(all_probs.cpu().numpy())
            refs.append(all_labels.cpu().numpy())

    preds = np.concatenate(preds)
    refs = np.concatenate(refs)
    
    if accelerator.is_main_process:
        preds = preds[:len(val_dataset)]
        refs = refs[:len(val_dataset)]
        
        avg_val_loss = val_loss / len(val_loader)
        
        from sklearn.metrics import roc_auc_score
        
        valid_aucs = []
        print("\nPer-class AUC:")
        for i, label in enumerate(LABELS):
            if len(np.unique(refs[:, i])) > 1:
                auc_score = roc_auc_score(refs[:, i], preds[:, i])
                valid_aucs.append(auc_score)
                print(f"  {label:15s}: {auc_score:.4f}")
        
        macro_auc = np.mean(valid_aucs) if valid_aucs else 0.0
        print(f"\n{'='*60}")
        print(f"Val Loss: {avg_val_loss:.4f} | Val AUC (macro): {macro_auc:.4f}")
        print(f"{'='*60}\n")
        
        if macro_auc > best_auc + MIN_DELTA:
            best_auc = macro_auc
            patience_counter = 0
            
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(
                best_model_path,
                save_function=accelerator.save
            )
            tokenizer.save_pretrained(best_model_path)
            print(f"✅ Best model saved! AUC: {best_auc:.4f}\n")
        else:
            patience_counter += 1
            print(f"⚠️  No improvement. Patience: {patience_counter}/{PATIENCE}\n")
        
        if patience_counter >= PATIENCE:
            print(f"🛑 Early stopping triggered after {epoch+1} epochs")
            break
    
    accelerator.wait_for_everyone()

if accelerator.is_main_process:
    print(f"\n{'='*60}")
    print(f"✅ Training Complete!")
    print(f"{'='*60}")
    print(f"Best Validation AUC: {best_auc:.4f}")
    print(f"Model saved at: {best_model_path}")
    print(f"{'='*60}\n")

🔧 GPUs available: 2
  GPU 0: Tesla T4
  GPU 1: Tesla T4


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...


Epoch 4/6
Train Loss: 0.0123



Per-class AUC:
  toxic          : 0.9853
  severe_toxic   : 0.9919
  obscene        : 0.9938
  threat         : 0.9813
  insult         : 0.9889
  identity_hate  : 0.9895

Val Loss: 0.0182 | Val AUC (macro): 0.9884

⚠️  No improvement. Patience: 1/2



Epoch 5/6: 100%|██████████| 7963/7963 [1:00:45<00:00,  2.18it/s, loss=0.00558]


Epoch 5/6
Train Loss: 0.0097



Per-class AUC:
  toxic          : 0.9854
  severe_toxic   : 0.9915
  obscene        : 0.9936
  threat         : 0.9812
  insult         : 0.9890
  identity_hate  : 0.9840

Val Loss: 0.0208 | Val AUC (macro): 0.9875

⚠️  No improvement. Patience: 2/2

🛑 Early stopping triggered after 5 epochs

✅ Training Complete!
Best Validation AUC: 0.9888
Model saved at: best_model



In [16]:
# ====================================================
# TEST INFERENCE
# ====================================================
model.eval()
test_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        outputs = model(**batch)
        logits = torch.sigmoid(outputs.logits)
        test_preds.append(accelerator.gather(logits).detach().cpu().numpy())

test_preds = np.concatenate(test_preds)
submission = pd.DataFrame(test_preds, columns=label_cols)
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved!")

Testing: 100%|██████████| 9573/9573 [19:13<00:00,  8.30it/s]


✅ submission.csv saved!
